<a href="https://colab.research.google.com/github/jovanape/Bitcoint-Price-Prediction/blob/main/Utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predvidjanje vrednosti Bitcoina pomocu GRU (Gated Recurrent Unit), RNN**

**Atributi:** Close

**Filtrirani podaci:** od 28.03.2021.

## **Ucitavanje biblioteka**

In [ ]:
# Ucitavanje biblioteka

import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU
#from keras.utils import np_utils
from keras.layers import GRU,CuDNNGRU
from keras.callbacks import CSVLogger, ModelCheckpoint
#import h5py
import os
from keras.losses import MeanSquaredError
#import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session

### **Podela skupa podataka**

In [ ]:
# Definicija funkcije za podelu podataka na train i test skup

def data_split(data, test_size = 0, days_to_predict = 0):

  if test_size == 0 and days_to_predict == 0:
    print('Uneti vrednost za jedan od parametara test_size ili days_to_predict.\n')
    return
  elif test_size == 0: # dato je days_to_predict
    test_limit = len(data) - prediction_days
  else: # dato je test_size ili oba parametra
    test_limit = len(data) - int(test_size * len(data))

  train = data[:test_limit]
  test = data[test_limit:]
  
  return train, test

### **Vizualizacija podataka**

In [ ]:
# Definicija funkcije za vizuelni prikaz train i test podataka

def plot_data(train_data = train['Close'], test_data = test['Close']):

  fig, ax = plt.subplots(1, figsize=(16, 9))
  ax.plot(train_data, label='trening', linewidth=2)
  ax.plot(test_data, label='test', linewidth=2)
  ax.set_ylabel('cena [USD]', fontsize=14)
  ax.set_title(label = 'Vizuelni prikaz skupova za trening i test', fontsize=18)
  ax.legend(loc='best', fontsize=18)

  #data = data.reset_index()
  #x = data.index
  #ax.set_xticks(range(87724))


## **Priprema trening skupa**

###**Filtriranje atributa train skupa**

In [1]:
# Izbacivanje nepotrebnih kolona (atributa) iz train skupa

def filter_train_set(train):
  train_ = train.drop(['Open', 'High', 'Low', 'Date', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price', 'Timestamp'], axis = 1)
  print('Filtriran trening skup:\n', train_.head())
  print('\nBroj atributa pre i nakon izbacivanja:\n', train.shape[1], train_.shape[1])
  return train_

###**Skaliranje train skupa**

In [ ]:
# Skaliranje trening skupa

def scale_train(train_):
  scaler = MinMaxScaler(feature_range=(0, 1))
  train_ = scaler.fit_transform(train_)
  return scaler, train_

array([[0.19947492],
       [0.21134984],
       [0.19379327],
       ...,
       [0.84805843],
       [0.84788789],
       [0.86661356]])

###**Odredjivanje ciljne promenljive na train skupu**


In [ ]:
# Odredjivanje ciljne promenljive na train skupu

def get_x_and_y():
  X_train = []
  y_train = []

  for i in range(60, train_.shape[0]):
    X_train.append(train_[i-60:i])
    y_train.append(train_[i,0])

  return X_train, y_train

## **Kompilacija i treniranje mreze**

In [2]:
# Definicija funkcije za kompilaciju i treniranje mreze
# Treniranje je moguce vrsiti sa ili bez validacije

def compile_and_fit(validation_data, validation_labels, optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse', 'mae'], X_train_ = X_train, y_train_ = y_train, epochs = 20, validation = 0.1, batch_size = 50, patience = 3):

  net.compile(loss = loss, optimizer = optimizer, metrics = metrics)

  callback = callbacks.EarlyStopping(monitor = 'loss', patience = patience)

  if validation == 0:
    history = net.fit(X_train_, y_train_, batch_size = batch_size, epochs = epochs, callbacks = [CSVLogger(output_file_name + '.csv', append = True), ModelCheckpoint('weights/' + output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor = 'val_loss', verbose = 1, mode = 'min'), callback])
  else:
    history = net.fit(X_train_, y_train_, batch_size = batch_size, validation_data = (validation_data, validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name + '.csv', append = True), ModelCheckpoint('weights/' + output_file_name + '-{epoch:02d}-{val_loss:.5f}.hdf5', monitor = 'val_loss', verbose = 1, mode = 'min'), callback])

  return history, net


NameError: ignored

## **Priprema test skupa**

### **Filtriranje atributa test skupa**

In [ ]:
# Izbacivanje nepotrebnih kolona (atributa) iz test skupa

def filter_test_set(test_):
  test_ = test.drop(['Open', 'High', 'Low', 'Date', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price', 'Timestamp'], axis = 1)
  return test_

### **Odredjivanje ciljne promenljive na test skupu**

In [ ]:
# Odredjivanje ciljne (target) promenljive na test skupu

def get_x_and_y():

  X_test = []
  y_test = []

  for i in range(60, test_.shape[0]):
    X_test.append(test_[i-60:i])
    y_test.append(test_[i,0])

  return X_test, y_test

## **Vizualizacija rezultata predvidjanja i metrika**

In [ ]:
# Vizuelni prikaz rezultata modela mreze

def plot_real_vs_predicted_data(test_set, predicted_set, caption):

  fig, ax = plt.subplots(1, figsize=(16, 9))
  ax.plot(test_set, label='Stvarna vrednost BTC', linewidth=2)
  ax.plot(predicted_set, label='Predvidjena vrednost BTC', linewidth=2)
  ax.set_xlabel('Timestamp', fontsize=14)
  ax.set_ylabel('Cena [USD]', fontsize=14)
  ax.set_title(label = caption, fontsize=18)
  ax.legend(loc='best', fontsize=18)

  # test = test.reset_index()
  # x = test.index


In [ ]:
# Vizuelni prikaz gubitka, MSE i MAE tokom treniranja mreze


# Vizuelni prikaz gubitka

def plot_loss(history_):

  epochs = range(len(history_.history['loss'])) # broj epoha

  plt.title('Vizuelni prikaz gubitka (loss) i MSE (mse) mreze')

  plt.subplot(211)
  
  plt.plot(epochs, history_.history['loss'], label='Gubitak (loss) tokom treniranja')

  if 'val_loss' in history_.history:
    plt.plot(epochs, history_.history['val_loss'], label='Gubitak (val_loss) tokom validacije')
  
  plt.legend()


# Vizuelni prikaz MSE

def plot_mse(history_):

  epochs = range(len(history_.history['mse'])) # broj epoha

  plt.title('Vizuelni prikaz srednjekvadratne greske (MSE) mreze')

  plt.subplot(212)

  plt.plot(epochs, history_.history['mse'], label='MSE (mse) tokom treniranja')
  
  if 'val_mse' in history_.history:
    plt.plot(epochs, history_.history['val_mse'], label='MSE (val_mse) tokom validacije')
  
  plt.legend()

  plt.show()


# Vizuelni prikaz MAE

def plot_mae(history_):

  epochs = range(len(history_.history['mae'])) # broj epoha
  plt.title('Vizuelni prikaz MAE (mae')
  plt.xlabel('broj epohe')
  plt.ylabel('MAE')
  
  plt.plot(epochs, history_.history['mae'], label='MAE (mae) tokom treniranja')

  if 'val_mae' in history_.history:
    plt.plot(epochs, history_.history['val_mae'], label='MAE (val_mae) tokom treniranja sa validacijom')

  plt.legend(loc = 'best')

  plt.show()